In [1]:
import json
import requests
from statistics import mean 
import operator

## Data Cleaning and Preparation

In [2]:
def return_url_params(var_url,player_id,start_date,year,oponent_team_id,venue_id,matchtype=None):
    year_wise_url = var_url+"api/year_wise_playermatchstats/"
    year_wise_params = {'player_id':player_id,'start_date':start_date,'year':year,'matchtype':matchtype,'lr':matchtype,'hr':matchtype}
    opponent_wise_url = var_url + 'api/opponent_wise_playermatchstats/'
    opponent_wise_params = {'player_id':player_id,'team_id':oponent_team_id,'start_date':start_date,'matchtype':matchtype,'lr':matchtype,'hr':matchtype}
    venue_wise_url = var_url + "api/venue_wise_playermatchstats/"
    venue_wise_params = {'player_id':player_id,'venue_id':venue_id,'start_date':start_date,'matchtype':matchtype,'lr':matchtype,'hr':matchtype}
    overall_url = var_url+'api/playermatchstats/'
    overall_params = {'player_id':player_id,'start_date':start_date,'lr':matchtype,'hr':matchtype}
    url_parms = {
        'v_u':venue_wise_url,
        'v_p':venue_wise_params,
        'o_u':opponent_wise_url,
        'o_p':opponent_wise_params,
        'y_u':year_wise_url,
        'y_p':year_wise_params,
        'ov_u':overall_url,
        'ov_p':overall_params
    }
    return url_parms

In [3]:
local_var = 'http://localhost:8000/'
global_var = 'https://win11.neolen.com/'

In [4]:
def get_stats(url,params):
    response = requests.request("GET", url, params = params)
    json_data = json.loads(response.text)
    return json_data

def return_int(d):
    try:
        d = int(d)
        return d
    except:
        return 0

In [5]:
def return_stats(json_data):     
    match_count = 0
    bowl_avg_list = []
    order_list = []
    bat_inning_count = 0
    bowl_inning_count = 0
    bat_avg_list = []
    list_50 = []
    list_100 = []
    list_5w = []
    list_3w = []
    for m in json_data:
        scorecard = m.get('scorecard',[])
        match_count += 1
        bowling_avg = 0 
        batting_avg = 0
        for s in scorecard:
            bat = s.get('match_batting_inning')
            bowl = s.get('match_bowling_inning')
            
            bat = bat[0] if len(bat) != 0 else None
                
            if bat:
                
                order = bat.get('order')
                run = return_int(bat.get('runs'))
                f_w = bat.get('fall_of_wicket_over')
                how_out = bat.get('how_out')

                #calculate batting average
                if f_w != '' and how_out != '':
                    bat_avg_list.append(float(run))
                elif how_out != "not out" and how_out != 'retired hurt':
                    bat_avg_list.append(float(run))
                else:
                    bat_avg_list.append(0.0)
                #end

                #calculate 50
                list_50.append(1 if run >= 50 and run < 100 else 0)
                #end

                #calculate 100
                list_100.append(1 if run>=100 else 0)
                #end
                
                bat_inning_count+=1
                order_list.append(order)

            bowl = bowl[0] if len(bowl) != 0 else None

            if bowl:
                
                run_conceded = return_int(bowl.get('run_conceded',0))
                wickets = return_int(bowl.get('wickets',0))
                
                #calculate bowling average
                if wickets != 0:
                    bowling_avg = run_conceded/wickets
                    bowl_avg_list.append(bowling_avg)
                else:
                    bowl_avg_list.append(0.0)
                #end
                
                #5 and 3 w
                list_3w.append(1 if wickets >= 3 and wickets < 5 else 0)
                list_5w.append(1 if wickets >= 5 else 0)
                #end
                
                bowl_inning_count+=1

#     print(match_count,bat_avg_list,bowl_avg_list,list_100,list_50,list_3w,list_5w,order_list,sep = '\n')
    bat_data = {
        'total_match':match_count,
        'innings':bat_inning_count,
        'avg':mean(bat_avg_list) if len(bat_avg_list) != 0 else 0,
        '50': sum(list_50),
        'order':mean(order_list) if len(order_list) != 0 else 0,
        '100':sum(list_100),
    }
    bowl_data = {
        'total_match':match_count,
        'innings': bowl_inning_count,
        'avg': mean(bowl_avg_list) if len(bowl_avg_list) != 0 else 0,
        '5w': sum(list_5w),
        '3w': sum(list_3w),
    }
    return bat_data , bowl_data

In [6]:
def batting_score(data,MAX_BATTING_AVG=250,MAX_50_100=30.0): #20*49+5*96
    B = data['avg']/MAX_BATTING_AVG
    C = (20*data['100'] + 5*data['50'])/data['innings'] if data['innings'] != 0 else 0
    D = C/MAX_50_100
    E = 0.4*D + 0.6*B
    return E
def bowling_score(data,MAX_BOWL_AVG=150,MAX_5_3=10.0):
    H = data['avg']/MAX_BOWL_AVG
    F = (20*data['5w'] + 5*data['3w'])/data['innings'] if data['innings'] != 0 else 0
    G = F/MAX_5_3
    I = 0.4*G + 0.6*H
    return I

In [7]:
def total_batting_score(data,player_stats):
    overall = player_stats['overall'][0]
    A = overall['innings']/overall['total_match'] if overall['total_match'] != 0 else 0
    E = data['overall']
    R = data['venue']
    P = data['year']
    Q = data['oponent']
    L = data['last_5']
    ans = A*((0.25*E) + (0.15*P) + (0.1*(Q+R))+(0.5*L))
    return round(ans,4)
def total_bowling_score(data,player_stats):
    overall = player_stats['overall'][1]
    V = overall['innings']/overall['total_match'] if overall['total_match'] != 0 else 0
    I = data['overall']
    S = data['year']
    T = data['oponent']
    U = data['venue']
    L1 = data['last_5']
    ans = V*((0.25*I) +(0.15*S) + (0.1 *
    (T+U) )+(0.5*L1))
    return round(ans,4)

In [8]:
def returnplayerusing_score(bat_score,bowl_score,allrounder_score):
    
    all_len = len(allrounder_score)
    bat_player_len = 0
    bowl_player_len = 0
    playing_11 = []
    if all_len == 3:
        bat_player_len = 4
        bowl_player_len = 4
    elif all_len < 3:
        bat_player_len = 5
        bowl_player_len = 11 - all_len - 5
    else:
        all_len = 3
        bat_player_len = 4
        bowl_player_len = 4
    for x in allrounder_score:
        playing_11.append(x[1])
        all_len -= 1
        if all_len == 0:
            break
    for y in bat_score:
        playing_11.append(y[1])
        bat_player_len-=1
        if bat_player_len == 0:
            break
    for y in bowl_score:
        playing_11.append(y[1])
        bowl_player_len-=1
        if bowl_player_len == 0:
            break
    
    return playing_11

In [9]:
def accuracy(player1 , player2):
    ans  = 0
    for p in player1:
        if p in player2:
            ans+=1
    return (ans/11)*100

In [26]:
def get_series_player(url,params):
    series_data = get_stats(local_var + url,params)
    series_data = series_data['results']
    print(len(series_data))
    team_players = {}
    for m in series_data:
        url_match_player = 'api/permatchplayer/'
        players_team1_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team1_id']})['results'][0]['players']
        players_team2_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team2_id']})['results'][0]['players']
        for p in players_team1_match:
            if team_players.get(m['team1_id']) == None:
                team_players[m['team1_id']] = set()
            team_players[m['team1_id']].add(p)
        for p in players_team2_match:
            if team_players.get(m['team2_id']) == None:
                team_players[m['team2_id']] = set()
            team_players[m['team2_id']].add(p)
    return team_players,series_data

In [11]:
def displayOfseriesresult(accuracy_of_matches):
    for x in accuracy_of_matches:
        print(x[2],x[0]," --> ",x[1])

In [12]:
# url_match = 'api/peryearmatch/'
# matches_19 = get_stats(local_var+url_match,{'year':2019})['results']

In [13]:
def player_stats_score(m,teamplayers,matchtype):
    bat_score_player_id = {}
    bow_score_player_id = {}
    bat_score = {}
    bowl_score = {}
    for p in teamplayers:

        batting_score_id = {}
        bowling_score_id = {}
        player_all_stats = return_url_params(local_var,p,m['start_date'],2019,m['team2_id'],m['venue_id'],matchtype=matchtype)

        #overallstats and last 5 match stats
        overall_stats = return_stats(get_stats(player_all_stats['ov_u'],player_all_stats['ov_p']).get('result',{}))
        if len(overall_stats) < 5:
            h = len(overall_stats)
        elif len(overall_stats) == 0:
            h = 0
        else:
            h = 5
        if h != 0:
            last_5_stats = overall_stats[:h]
        #end

        player_stats = {
            'year': return_stats(get_stats(player_all_stats['y_u'],player_all_stats['y_p'])['result']),
            'oponent': return_stats(get_stats(player_all_stats['o_u'],player_all_stats['o_p'])['result']),
            'venue': return_stats(get_stats(player_all_stats['v_u'],player_all_stats['v_p'])['result']),
            'overall': overall_stats,
            'last_5': last_5_stats
        }
        batting_score_id = {
            'overall' :batting_score(player_stats['overall'][0]),
             'venue'  :batting_score(player_stats['venue'][0]),
            'oponent':batting_score(player_stats['oponent'][0]),
             'year'   :batting_score(player_stats['year'][0]),
            'last_5': batting_score(player_stats['last_5'][0])
        }
        bowling_score_id = {
            'overall' :bowling_score(player_stats['overall'][1]),
             'venue'  :bowling_score(player_stats['venue'][1]),
            'oponent':bowling_score(player_stats['oponent'][1]),
             'year'   :bowling_score(player_stats['year'][1]),
            'last_5': bowling_score(player_stats['last_5'][1])
        }
        bat_s = total_batting_score(batting_score_id,player_stats)
        bowl_s = total_bowling_score(bowling_score_id,player_stats)
        bat_score_player_id[p] = bat_s
        bat_score[bat_s] = p
        bow_score_player_id[p] = bowl_s
        bowl_score[bowl_s] = p

    return bat_score,bowl_score,bat_score_player_id,bow_score_player_id

In [51]:
def playing11_model(series_id,matchtype):
    
    accuracy_of_matches = []
    print()
    print('START --> ',"%"*100)
    print()
    teamplayers , matches_perSeries = get_series_player('api/matchperyearOrseries/',{'series_id':series_id,'lr':matchtype,'hr':matchtype})    
    print(' players of teams in this series {} is {}'.format(series_id,teamplayers))
    print()
    for m in matches_perSeries[:20]:
        url_match_player = 'api/permatchplayer/'
        players_team1_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team1_id']})['results'][0]['players']
        players_team2_match = get_stats(local_var+url_match_player,{'match_id':m['id'],'team_id':m['team2_id']})['results'][0]['players']

        for t in [(teamplayers[m['team1_id']],players_team1_match,m['team1_id']),(teamplayers[m['team2_id']],players_team2_match,m['team2_id'])]:
            print('%'*100)
            print(m['description'])
            print('team id -> {}'.format(t[2]))
            bat_score , bowl_score , bat_score_player_id , bow_score_player_id = player_stats_score(m,t[0],matchtype)  

            all_rounder_score_player = {}
            #calculate the all rounder using threeshold
            for p in t[0]:
                if bat_score_player_id.get(p) != None and bow_score_player_id.get(p) != None:
                    if bat_score_player_id[p] <= 0.09 and bat_score_player_id[p] >= 0.09 and bow_score_player_id[p] <= 0.09 and bow_score_player_id[p] >= 0.02:
                        avg_score = (bat_score_player_id[p] + bow_score_player_id[p])/2
                        all_rounder_score_player[round(avg_score,4)] = p
                        del bat_score[bat_score_player_id[p]]
                        del bowl_score[bow_score_player_id[p]]
                        del bat_score_player_id[p]
                        del bow_score_player_id[p]
            #end

            #sort the score
            bat_score = sorted(bat_score.items(), key=operator.itemgetter(0),reverse=True)
            bowl_score = sorted(bowl_score.items(), key=operator.itemgetter(0),reverse=True)
            all_rounder_score_player = sorted(all_rounder_score_player.items(), key=operator.itemgetter(0),reverse=True)
            #end

            playing_11 = returnplayerusing_score(bat_score,bowl_score,all_rounder_score_player)

            print('batting score of player - >' , bat_score)
            print('bowling score of player - >' , bowl_score)
            print('allrounder score of player - >' , all_rounder_score_player)
            print()
            print('len of batsmen -> {} , len of bolwer - > {} , len of allrounder -> {} '.format(len(bat_score),len(bowl_score),len(all_rounder_score_player)))
            print("actual playing 11 --> ",t[1])
            print('predictive playing 11 of team {} - >'.format(t[2]),playing_11)
            acc = accuracy(playing_11,t[1])
            accuracy_of_matches.append((t[2],acc,m['description']))
            print('accuracy of predictive playing 11 of team {} -> '.format(t[2]),acc)
            print('%'*100)
    print("END --> ","%"*100)
    return accuracy_of_matches

In [15]:
series = [253,36,51,57,70,120,31,168,196,225,231,224,223,241]
# series = [31]

In [16]:
all_series_accuracy = []
for x in series:
    accuracyOfMatch = playing11_model(x,50.0)
    all_series_accuracy.append(accuracyOfMatch)
    displayOfseriesresult(accuracyOfMatch)


START -->  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

 players of teams in this series 253 is {85: {224, 962, 963, 4190, 1097, 1677, 1678, 1679, 113, 3989, 1687, 6617, 956, 958, 959}, 3: {32, 33, 672, 4192, 677, 713, 683, 716, 717, 719, 720, 691, 725, 726}}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
5th ODI (D/N), Australia tour of India at Delhi, Mar 13 2019
team id -> 85
len of batsmen -> 14 , len of bolwer - > 12 , len of allrounder -> 1 
actual playing 11 -->  [224, 962, 963, 1097, 1677, 1678, 1679, 113, 3989, 958, 959]
predictive playing 11 of team 85 - > [1097, 1677, 113, 958, 959, 4190, 963, 1679, 1687, 962, 3989]
accuracy of predictive playing 11 of team 85 ->  81.81818181818183
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

len of batsmen -> 13 , len of bolwer - > 12 , len of allrounder -> 2 
actual playing 11 -->  [2721, 2562, 2722, 676, 741, 2723, 14, 15, 17, 658, 19]
predictive playing 11 of team 2 - > [2770, 19, 2721, 15, 17, 2722, 658, 2771, 2723, 741, 2562]
accuracy of predictive playing 11 of team 2 ->  81.81818181818183
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Tour Match, England tour of South Africa at Paarl, Feb 1 2020
team id -> 201
len of batsmen -> 12 , len of bolwer - > 10 , len of allrounder -> 1 
actual playing 11 -->  [1153, 2405, 1895, 1896, 2766, 2767, 2768, 2769, 2611, 2612, 2615, 2396, 1822]
predictive playing 11 of team 201 - > [2766, 2612, 2611, 2396, 1822, 1895, 1896, 1153, 2615, 2405, 2769]
accuracy of predictive playing 11 of team 201 ->  100.0
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

 players of teams in this series 57 is {1: {2720, 706, 6, 1768, 9, 11, 1805, 1775, 1808, 1138, 1781, 1786, 1787, 1791}, 3: {32, 673, 674, 677, 683, 720, 1201, 30, 725, 23, 25, 26, 732, 702}}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
3rd ODI, Australia tour of South Africa at Potchefstroom, Mar 7 2020
team id -> 1
len of batsmen -> 13 , len of bolwer - > 9 , len of allrounder -> 1 
actual playing 11 -->  [706, 6, 1768, 9, 11, 1805, 1808, 1781, 1786, 1787, 1791]
predictive playing 11 of team 1 - > [1781, 6, 1787, 1768, 1138, 1791, 1808, 2720, 1775, 9, 1805]
accuracy of predictive playing 11 of team 1 ->  72.72727272727273
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
3rd ODI, Australia tour of South Africa at Potchefstroom, Mar 7 2020
team id -> 3
len of batsmen -> 12 , len o

len of batsmen -> 11 , len of bolwer - > 8 , len of allrounder -> 5 
actual playing 11 -->  [1183, 577, 613, 637, 646, 588, 627, 628, 605, 606, 639]
predictive playing 11 of team 185 - > [637, 605, 628, 614, 613, 639, 627, 588, 646, 3267, 599]
accuracy of predictive playing 11 of team 185 ->  72.72727272727273
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
1st ODI (D/N), Bangladesh tour of Sri Lanka at Colombo (RPS), Jul 26 2019
team id -> 86
len of batsmen -> 12 , len of bolwer - > 11 , len of allrounder -> 5 
actual playing 11 -->  [512, 1442, 2565, 966, 967, 3465, 971, 974, 975, 1688, 507]
predictive playing 11 of team 86 - > [972, 507, 971, 966, 493, 512, 1442, 974, 1255, 975, 2567]
accuracy of predictive playing 11 of team 86 ->  63.63636363636363
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

 players of teams in this series 168 is {184: {644, 1189, 3784, 2985, 2987, 459, 621, 622, 3345, 3352, 3609, 2555, 2556}, 86: {1249, 1442, 482, 965, 966, 968, 3465, 970, 972, 493, 1229, 975, 973, 1689}}

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
3rd ODI (D/N), Sri Lanka tour of Pakistan at Karachi, Oct 2 2019
team id -> 184
len of batsmen -> 11 , len of bolwer - > 10 , len of allrounder -> 2 
actual playing 11 -->  [644, 1189, 2985, 459, 2987, 621, 622, 3345, 3352, 3609, 2555]
predictive playing 11 of team 184 - > [1189, 2556, 459, 3784, 3345, 2985, 2555, 622, 644, 621, 3352]
accuracy of predictive playing 11 of team 184 ->  81.81818181818183
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
3rd ODI (D/N), Sri Lanka tour of Pakistan at Karachi, Oct 2 2019
team id -> 86
len of ba

KeyError: 0.0545

In [17]:
for a in all_series_accuracy:
    print("%"*100)
    displayOfseriesresult(a)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
5th ODI (D/N), Australia tour of India at Delhi, Mar 13 2019 85  -->  81.81818181818183
5th ODI (D/N), Australia tour of India at Delhi, Mar 13 2019 3  -->  81.81818181818183
4th ODI (D/N), Australia tour of India at Mohali, Mar 10 2019 85  -->  81.81818181818183
4th ODI (D/N), Australia tour of India at Mohali, Mar 10 2019 3  -->  81.81818181818183
3rd ODI (D/N), Australia tour of India at Ranchi, Mar 8 2019 85  -->  72.72727272727273
3rd ODI (D/N), Australia tour of India at Ranchi, Mar 8 2019 3  -->  72.72727272727273
1st ODI (D/N), Australia tour of India at Hyderabad (Deccan), Mar 2 2019 85  -->  72.72727272727273
1st ODI (D/N), Australia tour of India at Hyderabad (Deccan), Mar 2 2019 3  -->  81.81818181818183
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
3rd ODI, England tour of South Africa at Johannesburg, Feb 9 2020 1  -->

In [ ]:
# team1_Acc = []
# team2_ACC = []
# team = []
# for x in accuracy_of_matches:
#     if x[0] not in team:
#         team.append(x[0])
# total_acc_team1 = 0
# total_acc_team2 = 0
# for x in accuracy_of_matches:
#     if team[0] == x[0]:
#         total_acc_team1+=x[1]
#     if team[1] == x[0]:
#         total_acc_team2+=x[1]
# total_acc_team1 = total_acc_team1/3
# total_acc_team2 = total_acc_team2/3
# print('total acc of team1 is --> ',total_acc_team1)
# print('total acc of team2 is --> ',total_acc_team2)

# IPL 11 Player Selection

In [ ]:
# matchtype = 20.0
# teamplayers , matches_perSeries = get_ipl_series_player('api/matchperyearOrseries/',{'series_id':266,'lr':matchtype,'hr':matchtype})

In [52]:
accuracyOfMatch_ipl = playing11_model(266,20.0)


START -->  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

60
 players of teams in this series 266 is {681: {6, 781, 1677, 3990, 664, 2329, 1055, 162, 7971, 1068, 963, 6605, 974, 1103, 2773, 984, 989, 991, 4192, 372, 373, 3189}, 685: {386, 4, 1678, 1690, 2844, 2720, 5025, 5287, 8361, 8362, 41, 581, 5829, 6617, 4190, 224, 8677, 114, 756, 377}, 684: {772, 10, 3603, 3234, 44, 172, 695, 185, 957, 959, 6848, 207, 80, 723, 990, 353, 113, 369, 629, 120}, 687: {771, 3212, 17, 3989, 1686, 23, 36, 165, 687, 1083, 6851, 584, 1097, 72, 1102, 1104, 81, 1747, 1749, 4715, 366, 3695, 113}, 682: {5511, 14, 660, 6943, 1062, 3115, 6830, 3254, 575, 320, 962, 1734, 88, 734, 354, 362, 3695, 368, 115, 6015}, 683: {1679, 19, 1182, 2338, 1064, 1070, 8625, 178, 5299, 182, 956, 6849, 706, 1735, 1095, 6527, 987, 611, 1124, 4351}, 680: {132, 8341, 1687, 1063, 40, 42, 318, 958, 960, 961, 964, 6856, 713, 2770, 722, 986, 350, 1762, 1768, 2286, 1911}, 686: {2819, 

batting score of player - > [(0.0884, 984), (0.0828, 6), (0.0781, 1677), (0.0671, 1055), (0.0587, 373), (0.0552, 3189), (0.0438, 989), (0.0402, 372), (0.0265, 3990), (0.0241, 1103), (0.0178, 664), (0.0067, 2773), (0.0033, 6605), (0.003, 7971), (0.0027, 974), (0.0022, 2329), (0.002, 781), (0.0018, 4192), (0.0015, 1068), (0.0002, 963)]
bowling score of player - > [(0.2105, 991), (0.1152, 781), (0.1049, 974), (0.0795, 4192), (0.0773, 963), (0.0674, 6605), (0.0652, 664), (0.0619, 3990), (0.061, 2329), (0.051, 1103), (0.0507, 1068), (0.0458, 989), (0.0451, 7971), (0.0381, 2773), (0.0208, 372), (0.018, 3189), (0.0142, 162), (0.007, 1677), (0.0024, 373), (0.0, 984)]
allrounder score of player - > []

len of batsmen -> 20 , len of bolwer - > 20 , len of allrounder -> 0 
actual playing 11 -->  [963, 6, 1677, 974, 1103, 373, 2773, 3990, 2329, 989, 1055]
predictive playing 11 of team 681 - > [984, 6, 1677, 1055, 373, 991, 781, 974, 4192, 963, 6605]
accuracy of predictive playing 11 of team 681 ->

batting score of player - > [(0.1162, 23), (0.0829, 36), (0.0827, 81), (0.0795, 113), (0.072, 17), (0.0696, 1686), (0.0689, 771), (0.0671, 72), (0.0613, 3212), (0.0419, 4715), (0.0351, 584), (0.0346, 1097), (0.0339, 1104), (0.0337, 1102), (0.0169, 3695), (0.0053, 687), (0.0029, 3989), (0.0026, 165), (0.002, 1747), (0.0011, 6851), (0.0007, 1749), (0.0, 366)]
bowling score of player - > [(0.1294, 1083), (0.1198, 687), (0.0988, 6851), (0.0857, 3989), (0.0816, 1749), (0.0807, 4715), (0.0765, 366), (0.072, 3695), (0.0675, 165), (0.0602, 584), (0.0373, 1747), (0.0259, 1104), (0.0113, 1097), (0.0086, 1102), (0.0079, 36), (0.0018, 113), (0.0012, 1686), (0.0001, 771), (0.0, 81)]
allrounder score of player - > []

len of batsmen -> 22 , len of bolwer - > 19 , len of allrounder -> 0 
actual playing 11 -->  [771, 36, 584, 1097, 3212, 366, 687, 1104, 3989, 1686, 1083]
predictive playing 11 of team 687 - > [23, 36, 81, 113, 17, 1083, 687, 6851, 3989, 1749, 4715]
accuracy of predictive playing 11 of 

batting score of player - > [(0.1086, 5287), (0.0749, 581), (0.0746, 4), (0.0662, 114), (0.0607, 5829), (0.057, 8362), (0.0486, 6617), (0.0413, 4190), (0.0344, 224), (0.0247, 1690), (0.0229, 41), (0.0155, 1678), (0.0098, 8361), (0.0042, 756), (0.0033, 5025), (0.0015, 8677), (0.0011, 377), (0.0009, 2844), (0.0, 2720)]
bowling score of player - > [(0.0974, 5025), (0.097, 1690), (0.0881, 756), (0.0877, 8677), (0.0827, 2844), (0.0736, 377), (0.0566, 386), (0.0525, 41), (0.0498, 1678), (0.0494, 8361), (0.0483, 581), (0.0362, 2720), (0.0188, 4), (0.0095, 8362), (0.0012, 224), (0.0, 114)]
allrounder score of player - > []

len of batsmen -> 19 , len of bolwer - > 16 , len of allrounder -> 0 
actual playing 11 -->  [224, 5025, 4, 581, 8361, 8362, 1678, 6617, 1690, 2844, 4190]
predictive playing 11 of team 685 - > [5287, 581, 4, 114, 5829, 5025, 1690, 756, 8677, 2844, 377]
accuracy of predictive playing 11 of team 685 ->  45.45454545454545
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

batting score of player - > [(0.102, 958), (0.0736, 1911), (0.0676, 350), (0.0614, 2770), (0.057, 986), (0.0423, 961), (0.0417, 1063), (0.0406, 40), (0.0362, 713), (0.0361, 318), (0.0343, 1768), (0.0326, 8341), (0.0117, 960), (0.0102, 6856), (0.0068, 42), (0.0024, 1762), (0.002, 722), (0.0014, 132), (0.0005, 1687), (0.0, 2286)]
bowling score of player - > [(0.1097, 132), (0.0845, 722), (0.0818, 42), (0.0816, 1762), (0.0792, 1687), (0.0708, 6856), (0.0661, 964), (0.066, 960), (0.0646, 2286), (0.0488, 713), (0.0482, 2770), (0.0212, 40), (0.0145, 318), (0.0035, 1063), (0.0011, 958), (0.0008, 1768), (0.0, 1911)]
allrounder score of player - > []

len of batsmen -> 20 , len of bolwer - > 17 , len of allrounder -> 0 
actual playing 11 -->  [960, 961, 350, 1762, 964, 1063, 1768, 713, 1911, 1687, 958]
predictive playing 11 of team 680 - > [958, 1911, 350, 2770, 986, 132, 722, 42, 1762, 1687, 6856]
accuracy of predictive playing 11 of team 680 ->  45.45454545454545
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

batting score of player - > [(0.0979, 958), (0.0805, 1911), (0.0713, 350), (0.0627, 2770), (0.0575, 986), (0.0438, 961), (0.0414, 1063), (0.0412, 713), (0.0403, 40), (0.0401, 318), (0.0281, 8341), (0.0266, 1768), (0.0117, 960), (0.0116, 6856), (0.0061, 42), (0.0024, 722), (0.0023, 1762), (0.0012, 132), (0.0004, 1687), (0.0, 2286)]
bowling score of player - > [(0.1108, 132), (0.0893, 42), (0.0845, 1687), (0.0825, 722), (0.0806, 1762), (0.0752, 6856), (0.066, 960), (0.0626, 961), (0.0602, 964), (0.0569, 2286), (0.0531, 713), (0.0494, 2770), (0.023, 40), (0.0148, 318), (0.0044, 1063), (0.0011, 958), (0.0008, 1768), (0.0, 1911)]
allrounder score of player - > []

len of batsmen -> 20 , len of bolwer - > 18 , len of allrounder -> 0 
actual playing 11 -->  [318, 961, 1762, 964, 713, 42, 1911, 2770, 958, 1687, 350]
predictive playing 11 of team 680 - > [958, 1911, 350, 2770, 986, 132, 42, 1687, 722, 1762, 6856]
accuracy of predictive playing 11 of team 680 ->  63.63636363636363
%%%%%%%%%%%%%%

In [53]:
displayOfseriesresult(accuracyOfMatch_ipl)

Final (N), Indian Premier League at Hyderabad (Deccan), May 12 2019 681  -->  63.63636363636363
Final (N), Indian Premier League at Hyderabad (Deccan), May 12 2019 685  -->  63.63636363636363
Qualifier 2 (N), Indian Premier League at Visakhapatnam, May 10 2019 685  -->  63.63636363636363
Qualifier 2 (N), Indian Premier League at Visakhapatnam, May 10 2019 684  -->  63.63636363636363
Eliminator (N), Indian Premier League at Visakhapatnam, May 8 2019 684  -->  63.63636363636363
Eliminator (N), Indian Premier League at Visakhapatnam, May 8 2019 687  -->  45.45454545454545
Qualifier 1 (N), Indian Premier League at Chennai, May 7 2019 681  -->  54.54545454545454
Qualifier 1 (N), Indian Premier League at Chennai, May 7 2019 685  -->  54.54545454545454
56th Match (N), Indian Premier League at Mumbai, May 5 2019 681  -->  72.72727272727273
56th Match (N), Indian Premier League at Mumbai, May 5 2019 682  -->  72.72727272727273
55th Match (D/N), Indian Premier League at Mohali, May 5 2019 683  -

In [54]:
!git add .

In [55]:
# !git commit -m " model changed little bit concept"

[master ee2417f]  model changed little bit concept
 1 file changed, 434 insertions(+), 137 deletions(-)


In [56]:
# !git push origin master

Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 8.55 KiB | 515.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To github.com:SabiNinza/Machine-Learning.git
   bf298a8..ee2417f  master -> master
